In [ ]:
# Imports
import numpy as np
import pandas as pd
import stan
import arviz as az
import matplotlib.pyplot as plt

import nest_asyncio
nest_asyncio.apply()

# Load data
url = 'https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv'
data = pd.read_csv(url)

# Preprocess and standardize data
data['bmi_std'] = (data['bmi'] - data['bmi'].mean()) / data['bmi'].std()
data['age_std'] = (data['age'] - data['age'].mean()) / data['age'].std()
data['children_std'] = (data['children'] - data['children'].mean()) / data['children'].std()
data['charges_std'] = (data['charges'] - data['charges'].mean()) / data['charges'].std()

# Define Stan model for multiple regression
model_code = """
data {
    int<lower=0> N;
    int<lower=0> K;
    matrix[N, K] X;
    vector[N] y;
    int<lower=0> N_test;
    matrix[N_test, K] X_test;
}
parameters {
    real alpha;
    vector[K] beta;
    real<lower=0> sigma;
}
model {
    // Priors
    alpha ~ normal(0, 10);
    beta ~ normal(0, 5);
    sigma ~ inv_gamma(2, 3);

    // Likelihood
    y ~ normal(X * beta + alpha, sigma);
}
generated quantities {
    array[N_test] real y_pred;
    for (i in 1:N_test) {
        y_pred[i] = normal_rng(X_test[i] * beta + alpha, sigma);
    }
}
"""

# Prepare data for Stan model
stan_data = {
    'N': 1000,
    'K': 3,
    'X': data[['bmi_std', 'age_std', 'children_std']][:1000].values,
    'y': data['charges_std'][:1000].values,
    'N_test': len(data) - 1000,
    'X_test': data[['bmi_std', 'age_std', 'children_std']][1000:].values
}

# Compile and fit Stan model
posterior = stan.build(model_code, data=stan_data, random_seed=42)
fit = posterior.sample(num_chains=4, num_samples=1000)



In [ ]:
# Diagnose and summarize
diagnostics = az.summary(fit)
diagnostics

In [ ]:
# analyze results with RMSE
y_all_range = max(data['charges_std'])-min(data['charges_std'])
y_all_norm = (data['charges_std']-min(data['charges_std']))/(max(data['charges_std'])-min(data['charges_std']))
y_all_norm_range = max(y_all_norm)-min(y_all_norm)

y_pred = np.mean(np.array(fit['y_pred']), axis = 1)
y = data['charges_std'][1000:].values

rmse = np.sqrt(np.mean((y - y_pred)**2))

print(f"RMSE of unnormalized data: {rmse}")

print(f"Range of unnormalized data: {y_all_range}")

# normalizing
y_norm = (y-min(data['charges_std']))/(max(data['charges_std'])-min(data['charges_std']))

y_pred_norm = (y_pred-min(data['charges_std']))/(max(data['charges_std'])-min(data['charges_std']))

rmse = np.sqrt(np.mean((y_norm - y_pred_norm)**2))

print(f"RMSE of normalized data: {rmse}")

print(f"Range of normalized data: {y_all_norm_range}")
